### FYS-STK4155 PROJECT 1

## **FUNCTIONS**

In [ ]:
import numpy as np

#### Franke's Function
Setting up Franke's function (as given in assignment text)

In [ ]:
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

#### Vandermonde Matrix (design matrix) of Degree $n$

In [ ]:
def VandermondeMatrix(x, y, n):
    X = np.c_[np.ones(len(x))]
    for i in range(1, n+1):
        # x-terms
        X = np.c_[X, x**(i)]
        # y-terms
        X = np.c_[X, y**(i)]
        # Cross terms
        for j in range(i-1, 0, -1):
            X = np.c_[X, (x**(j))*(y**(i-j))]
            
    return X

#### Ordinary Least-Squares

In [ ]:
def linreg_ols(X, z):
    
    # Solving for beta
    beta = np.linalg.inv(np.transpose(X).dot(X)).dot(np.transpose(X)).dot(z)
    
    y_predict_ols = X @ beta
    
    return beta, y_predict_ols

#### Ridge regression

In [ ]:
def ridgereg(X, z, lambda_ridge):

    # Solving for beta
    beta_ridge = np.linalg.inv(np.transpose(X).dot(X) + lambda_ridge*np.identity(X.shape[1])).dot(np.transpose(X)).dot(z)

    y_predict_ridge = X @ beta_ridge
    
    return beta_ridge, y_predict_ridge

#### Lasso regression

In [ ]:
from sklearn.linear_model import Lasso

def lassoreg(X, z, lambda_lasso):
    
    las = Lasso(alpha=lambda_lasso, fit_intercept = False)
    las.fit(X, z)
    
    beta = las.coef_#[:,np.newaxis]
    
    y_predict_lasso = X @ beta
    
    R2_lasso = las.score(X, z)
    
    return beta, y_predict_lasso, R2_lasso

#### $R^{2}$ function (taken from lecture notes)

The ideal fit gives $R^{2} = 1$

In [ ]:
def R2(y_data, y_model):
    return 1 - np.sum((y_data - y_model) ** 2) / np.sum((y_data - np.mean(y_data)) ** 2)

#### Mean Squared Error (MSE) function

The ideal fit gives $MSE = 0$

In [ ]:
def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

#### Variance

(Find formula from book)

In [ ]:
# Variance
def var(y_data, y_model):
    n = 20
    return np.sum((y_model - np.mean(y_model))**2) / n

#### Confidence interval

In [ ]:
def confidence_interval(X, beta, z_level, std):
    
    XtXi = np.linalg.inv(np.transpose(X).dot(X))

    num_beta = beta.shape[0]
    
    diagonals = np.zeros(num_beta)
    
    for j in range(num_beta):
        diagonals[j] = np.sqrt(XtXi[j][j])
        
    cint_upper = np.zeros(num_beta)
    cint_lower = np.zeros(num_beta)
    
    for i in range(num_beta):
        
        cint_upper[i] = beta[i] + z_level*std
        cint_lower[i] = beta[i] - z_level*std
        
    return cint_upper, cint_lower, diagonals

####  **END OF FUNCTIONS**

## **START OF PROGRAM**

In [ ]:
# GENERATING DATA
x = np.arange(0, 1, 0.05)
y = np.arange(0, 1, 0.05)

x, y = np.meshgrid(x, y)

z = FrankeFunction(x, y)

x = np.ravel(x)
y = np.ravel(y)
z = np.ravel(z)

z = z #+ 0.1 * np.random.randn(z.shape[0]) # noise level = 0.1

In [ ]:
# COMPARING WITH SCI-KIT LEARN

from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(X, z)
z_predict_scikit = X @ reg.coef_

R2_scikit.append(reg.score(X, z))

MSE_scikit.append(MSE(z, z_predict_scikit))

In [ ]:
from sklearn.linear_model import LinearRegression

R2_scores = list()
MSE_scores = list()
R2_scikit = list()
MSE_scikit = list()
for n in range(1,6):
    X = VandermondeMatrix(x, y, n)

    beta_ols, z_predict_ols = linreg_ols(X, z)
    

    R2_scores.append(R2(z, z_predict_ols))
    MSE_scores.append(MSE(z, z_predict_ols))
    
    # COMPARING WITH SCI-KIT LEARN

    reg = LinearRegression().fit(X, z)
    z_predict_scikit = X @ reg.coef_

    R2_scikit.append(reg.score(X, z))
    MSE_scikit.append(MSE(z, z_predict_scikit))
    
    print("n =", n, "\n")
    #print("Own code | scikit-learn")
    #print("R2 ols", R2(z, z_predict_ols), reg.score(X,z))
    #print("MSE ols", MSE(z, z_predict_ols), MSE(z, z_predict_scikit))
                      
                      
    lambdas = [0.1, 0.01, 0.001, 0.0001]
    
    for la in lambdas:
    #    beta_ridge, z_predict_ridge = ridgereg(X, z, lambda_ridge = la)
        #print("Lambda:", la)
        #print("R2 ridge", R2(z, z_predict_ridge))
        #print("MSE ridge", MSE(z, z_predict_ridge))
        beta_lasso, z_predict_lasso, R2_lasso = lassoreg(X, z, lambda_lasso = la)
        print("R2 lasso", R2_lasso, "lambda =", la)
        print("MSE lasso", MSE(z, z_predict_lasso), "lambda =", la)
        print("\n")
        
    #print("\n")
    #print("******************************")

    
"""
n = range(1,6)
#plt.scatter(n, R2_scores)
#print(MSE_scores)
#plt.scatter(n, MSE_scores, color='red')
#plt.show()

fig, ax1 = plt.subplots()

color = 'tab:cyan'
ax1.set_xlabel('Model complexity')
ax1.set_ylabel('R2', color=color)
ax1.plot(n, R2_scores, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:orange'
ax2.set_ylabel('MSE', color=color)  # we already handled the x-label with ax1
ax2.plot(n, MSE_scores, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.grid()
plt.show() 
"""

#### Confidence interval

In [ ]:
x = np.arange(0, 1, 0.05)
y = np.arange(0, 1, 0.05)

x, y = np.meshgrid(x, y)

z = FrankeFunction(x, y)

x = np.ravel(x)
y = np.ravel(y)
z = np.ravel(z)

z = z + 0.1 * np.random.randn(z.shape[0]) # noise level = 0.1

X = VandermondeMatrix(x, y, 5)

beta_ols, z_predict_ols = linreg_ols(X, z)
beta_ridge, z_predict_ridge = ridgereg(X, z, lambda_ridge = 0.001)
beta_lasso, z_predict_lasso, R2_lasso = lassoreg(X, z, lambda_lasso = 0.001)

In [ ]:
c_upper_ols, c_lower_ols, diag_ols = confidence_interval(X, beta_ols, 1.96, 0.1)

#for c in range(c_upper_ols.shape[0]):
#    print('$ \\beta_{c} $', '&', round(c_lower_ols[c], 5), '&', round(c_upper_ols[c],5), '&', "Var:", round(diag_ols[c], 5), '\\\\')

In [ ]:
c_upper_ridge, c_lower_ridge, diag_ridge = confidence_interval(X, beta_ridge, 1.96, 0.1)

#for c in range(c_upper_ridge.shape[0]):
#    print('$ \\beta_{c} $', '&', round(c_lower_ridge[c], 5), '&', round(c_upper_ridge[c],5), '&', "Var:", round(diag_ridge[c], 5), '\\\\')

In [ ]:
c_upper_lasso, c_lower_lasso, diag_lasso = confidence_interval(X, beta_lasso, 1.96, 0.1)

#for c in range(c_upper_lasso.shape[0]):
#    print('$ \\beta_{c} $', '&', round(c_lower_lasso[c], 5), '&', round(c_upper_lasso[c],5), '&', "Var:", round(diag_lasso[c], 5), '\\\\')

In [ ]:
%matplotlib inline
x_axis = np.arange(0,21,1)

plt.scatter(x_axis, c_upper_ols, label='Upper', color='cyan')
plt.scatter(x_axis, c_lower_ols, label='Lower', color='red')
plt.show()

In [ ]:
a = np.empty([50, 21])
a[0] = beta


In [ ]:
def bootstrap(X, z, statistic, method, number_of_bootstraps, la):
    stat = np.empty([number_of_bootstraps, X.shape[1]])
    max_idx = X.shape[0] # highest number of indices
    beta = np.empty([number_of_bootstraps, X.shape[1]])
    z_predict = np.empty([number_of_bootstraps, X.shape[0]])
    R2_lasso = np.zeros(number_of_bootstraps)
    
    np.random.seed(4155)
    
    for i in range(number_of_bootstraps):
           
        if method == 'OLS':
            idx = np.random.randint(0, max_idx, max_idx)
            beta[i], z_predict[i] = linreg_ols(X[idx], z[idx])
                            
        elif method == 'Ridge':
            idx = np.random.randint(0, max_idx, max_idx)
            beta[i], z_predict[i] = ridgereg(X[idx], z[idx], la)
            
        elif method == 'LASSO':
            idx = np.random.randint(0, max_idx, max_idx)
            beta[i], z_predict[i], R2_lasso[i] = lassoreg(X[idx], z[idx], la)
            
        else:
            break
         
        if statistic:
            stat[i] = statistic(beta)
        
    if not statistic:
        if method == 'OLS' or method == 'Ridge':
            return beta, z_predict
        if method == 'LASSO':
            return beta, z_predict, R2_lasso

In [ ]:
X = VandermondeMatrix(x, y, 5)

from sklearn.model_selection import train_test_split
X_train, X_test, z_train, z_test = train_test_split(X, z, test_size = 0.2)

In [ ]:
beta_bootstrap_ols, z_predict_boot_ols = bootstrap(X_train, z_train, statistic = False, method = 'OLS', number_of_bootstraps = 5000, la = None)
beta_bootstrap_ridge, z_predict_boot_ridge = bootstrap(X_train, z_train, statistic = False, method = 'Ridge', number_of_bootstraps = 5000, la = 0.01)
beta_bootstrap_lasso, z_predict_boot_lasso, R2_lasso = bootstrap(X_train, z_train, statistic = False, method = 'LASSO', number_of_bootstraps = 5000, la = 0.01)

In [ ]:
beta_mean_ols = np.mean(beta_bootstrap_ols, axis=0)
beta_mean_ridge = np.mean(beta_bootstrap_ridge, axis=0)
beta_mean_lasso = np.mean(beta_bootstrap_lasso, axis=0)

z_pred_ols = X_test @ beta_mean_ols
z_pred_ridge = X_test @ beta_mean_ridge
z_pred_lasso = X_test @ beta_mean_lasso

print('R2 \n')
print('OLS', R2(z_test, z_pred_ols), '\n RIDGE', R2(z_test, z_pred_ridge), '\n LASSO', R2(z_test, z_pred_lasso))
print('\n MSE \n')
print('OLS', MSE(z_test, z_pred_ols), '\n RIDGE', MSE(z_test, z_pred_ridge), '\n LASSO', MSE(z_test, z_pred_lasso))

In [ ]:
X = VandermondeMatrix(x, y, 1)
X.shape[1]

In [ ]:
degrees = np.arange(1, 10, 1)
R2_scores_ols = list()
R2_scores_ridge = list()
R2_scores_lasso = list()
MSE_scores_ols = list()
MSE_scores_ridge = list()
MSE_scores_lasso = list()
for n in degrees:
    X = VandermondeMatrix(x, y, n)

    from sklearn.model_selection import train_test_split
    X_train, X_test, z_train, z_test = train_test_split(X, z, test_size = 0.2)
    
    beta_bootstrap_ols, z_predict_boot_ols = bootstrap(X_train, z_train, statistic = False, method = 'OLS', number_of_bootstraps = 5000, la = None)
    beta_bootstrap_ridge, z_predict_boot_ridge = bootstrap(X_train, z_train, statistic = False, method = 'Ridge', number_of_bootstraps = 5000, la = 0.01)
    beta_bootstrap_lasso, z_predict_boot_lasso, R2_lasso = bootstrap(X_train, z_train, statistic = False, method = 'LASSO', number_of_bootstraps = 5000, la = 0.01)

    
    beta_mean_ols = np.mean(beta_bootstrap_ols, axis=0)
    beta_mean_ridge = np.mean(beta_bootstrap_ridge, axis=0)
    beta_mean_lasso = np.mean(beta_bootstrap_lasso, axis=0)

    z_pred_ols = X_test @ beta_mean_ols
    z_pred_ridge = X_test @ beta_mean_ridge
    z_pred_lasso = X_test @ beta_mean_lasso

    R2_scores_ols.append(R2(z_test, z_pred_ols))
    R2_scores_ridge.append(R2(z_test, z_pred_ridge))
    R2_scores_lasso.append(R2(z_test, z_pred_lasso))
    MSE_scores_ols.append(MSE(z_test, z_pred_ols))
    MSE_scores_ridge.append(MSE(z_test, z_pred_ridge))
    MSE_scores_lasso.append(MSE(z_test, z_pred_lasso))
    
    
    print('n:', n, '\n')
    print('R2 \n')
    print('OLS', R2(z_test, z_pred_ols), '\n RIDGE', R2(z_test, z_pred_ridge), '\n LASSO', R2(z_test, z_pred_lasso))
    print('\n MSE \n')
    print('OLS', MSE(z_test, z_pred_ols), '\n RIDGE', MSE(z_test, z_pred_ridge), '\n LASSO', MSE(z_test, z_pred_lasso))

In [ ]:
R2_scores_ols

In [ ]:
plt.plot(R2_scores_ols)
plt.show()

In [ ]:
fig, ax1 = plt.subplots()

color = 'tab:cyan'
ax1.set_xlabel('Complexity')
ax1.set_ylabel('R2', color=color)
ax1.plot(degrees, R2_scores_ols, color=color, ls = '-', label= 'OLS')
ax1.plot(degrees, R2_scores_ridge, color=color, ls=':', label = 'Ridge')
ax1.plot(degrees, R2_scores_lasso, color=color, ls = '-.', label = 'Lasso')
ax1.tick_params(axis='y', labelcolor=color)
plt.legend()

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:orange'
ax2.set_ylabel('MSE', color=color)  # we already handled the x-label with ax1
ax2.plot(degrees, MSE_scores_ols, color=color, ls = '-', label= 'OLS')
ax2.plot(degrees, MSE_scores_ridge, color=color, ls = ':', label= 'Ridge')
ax2.plot(degrees, MSE_scores_lasso, color=color, ls = '-.', label = 'Lasso')
ax2.tick_params(axis='y', labelcolor=color)
plt.legend()

fig.tight_layout()  # otherwise the right y-label is slightly clipped

plt.title('Model complexity (after 5000 bootstraps)')
plt.grid()
plt.show()